In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'] = 'Gulim'

import warnings
warnings.filterwarnings(action='ignore')

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
from workalendar.asia import SouthKorea
import pendulum

In [3]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

## 전처리
- 일자에서 월과 일을 분리
- 요일을 레이블 인코딩화(EDA로 요일의 중요도 순 파악)
- 월 별, 일 별 중식 석식 수요 차이 파악

In [4]:
train['월'] = pd.DatetimeIndex(train['일자']).month
test['월'] = pd.DatetimeIndex(test['일자']).month
train['주'] = pd.DatetimeIndex(train['일자']).week
test['주'] = pd.DatetimeIndex(test['일자']).week
train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

# train['중식참여율'] = train['중식계'] / train['식사가능자수']

In [5]:
month_rank4dinner = {
    1: 11,
    2: 2,
    3: 1,
    4: 4,
    5: 7,
    6: 6,
    7: 10,
    8: 8,
    9: 5,
    10: 3,
    11: 9,
    12: 12
}
train['월(석식)'] = train['월'].map(month_rank4dinner)
test['월(석식)'] = test['월'].map(month_rank4dinner)

month_rank4lunch = {
    1: 3,
    2: 1,
    3: 2,
    4: 6,
    5: 7,
    6: 8,
    7: 10,
    8: 9,
    9: 5,
    10: 4,
    11: 11,
    12: 12
}
train['월(중식)'] = train['월'].map(month_rank4lunch)
test['월(중식)'] = test['월'].map(month_rank4lunch)

weekday_rank4dinner = {
    '월': 1,
    '화': 2,
    '수': 4,
    '목': 3,
    '금': 5,
}

weekday_rank4lunch = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5,
}

train['요일(석식)'] = train['요일'].map(weekday_rank4dinner)
test['요일(석식)'] = test['요일'].map(weekday_rank4dinner)

train['요일(중식)'] = train['요일'].map(weekday_rank4lunch)
test['요일(중식)'] = test['요일'].map(weekday_rank4lunch)

In [6]:
rank = pd.DataFrame(range(1,53))
week_rank_lunch = pd.pivot_table(train,values='중식계',index='주').sort_values(by='중식계').reset_index().drop('중식계',axis=1)
week_rank_dinner = pd.pivot_table(train,values='석식계',index='주').sort_values(by='석식계').reset_index().drop('석식계',axis=1)


week_rank4lunch = {}
for i in range(len(rank)):
    week_rank4lunch[week_rank_lunch['주'][i]] = rank[0][i]


week_rank4dinner = {}
for i in range(len(rank)):
    week_rank4dinner[week_rank_dinner['주'][i]] = rank[0][i]
    
    
train['주(중식)'] = train['주'].map(week_rank4lunch)
test['주(중식)'] = test['주'].map(week_rank4lunch)

train['주(석식)'] = train['주'].map(week_rank4dinner)
test['주(석식)'] = test['주'].map(week_rank4dinner)

In [7]:
def is_holiday(date):
    holidays = list(map(str, pd.Series(np.array(SouthKorea().holidays(int(date[:4])))[:, 0])))
    
    yesterday = str(np.datetime64(date) - 1)
    tomorrow = str(np.datetime64(date) + 1)

    if tomorrow in holidays and yesterday in holidays:
        return 3
    if tomorrow in holidays:
        return 2
    elif yesterday in holidays:
        return 1
    else : 
        return 0

def week_of_month(x):
    dt = pendulum.parse(x)
    
    wom = dt.week_of_month
    if wom < 0:
        wom += 52
    return wom
    

df = pd.concat([train[['본사정원수', '일자']], test[['본사정원수', '일자']]])
df['년월'] = df['일자'].apply(lambda x : x[:7])
df = df[['년월', '본사정원수']].groupby(by=['년월'], as_index=False).mean()

def member_change(date):
    this_month = date[:7]
    last_month = str(np.datetime64(this_month) - 1)
    
    this_month_member = int(df[df['년월'] == this_month]['본사정원수'])
    last_month_member = int(df[df['년월'] == last_month]['본사정원수'])
    
    
    return  this_month_member - last_month_member

train['공휴일전후'] = train['일자'].apply(is_holiday)
test['공휴일전후'] = test['일자'].apply(is_holiday)

train['몇주차'] = train['일자'].apply(week_of_month)
test['몇주차'] = test['일자'].apply(week_of_month)

train = train[train['일자'] > '2016-03']
train['인원변화'] = train['일자'].apply(member_change)
test['인원변화'] = test['일자'].apply(member_change)

## 공휴일 변수 생성

In [8]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '월', '주', '일',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '식사가능자수', '월(석식)', '월(중식)',
       '요일(석식)', '요일(중식)', '주(중식)', '주(석식)', '공휴일전후', '몇주차', '인원변화'],
      dtype='object')

In [9]:
# 메뉴 변수 없이 사용할떄 해당 코드 사용['공휴일전후', '몇주차', '인원변화']

lunch_train = train[['공휴일전후', '몇주차', '인원변화', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]
lunch_test = test[['공휴일전후', '몇주차', '인원변화', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]

dinner_train= train[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]
dinner_test = test[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]

y_lunch = np.array(train[['중식계']])
y_dinner = np.array(train[['석식계']])

In [10]:
lunch_train.columns

Index(['공휴일전후', '몇주차', '인원변화', '요일(중식)', '월(중식)', '일', '주(중식)', '출근', '휴가비율',
       '출장비율', '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수', '본사시간외근무명령서승인건수'],
      dtype='object')

In [11]:
print(lunch_train.shape)
print(lunch_test.shape)
print(dinner_train.shape)
print(dinner_test.shape)

(1187, 16)
(50, 16)
(1187, 16)
(50, 16)


In [12]:
cat_features = [f for f in lunch_train.columns if lunch_train[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(lunch_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

Cat features are: []
[]


#### 분포 확인 및 분포 조정

# 중식 예측모델

In [17]:
import optuna
from catboost import CatBoostRegressor
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, KFold

In [18]:
# from automl_alex import LightGBMRegressor, CatBoostRegressor, AutoMLRegressor
from sklearn.metrics import mean_absolute_error as MAE
import sklearn

In [32]:
def objective_lunch(trial: Trial) -> float:    
    params_cat = {
        "random_state": 42,
        "learning_rate": 0.05,
        "n_estimators": 10000,
        "verbose" : 1,
        "objective" : "MAE",
        "eval_metric" : "MAE",
        "max_depth": trial.suggest_int("max_depth", 1, 16),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.8, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }
    
    n_fold = 4
    kf = KFold(n_splits=n_fold, random_state=42, shuffle=True)
    mae_log = 0
    
    for train_index, test_index in kf.split(lunch_train):
        X_train, X_valid = lunch_train.iloc[train_index], lunch_train.iloc[test_index]
        y_train, y_valid = y_lunch[train_index], y_lunch[test_index]

        model = CatBoostRegressor(**params_cat)

        evaluation = [(X_train, y_train), (X_valid, y_valid)]

        model.fit(X_train, y_train,
                  eval_set=evaluation, 
                  early_stopping_rounds=100,
                  verbose=0)

        pred = model.predict(X_valid)
        mae= MAE(y_valid, pred)   
        
        mae_log += mae/n_fold
    
    return mae

In [54]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="cat_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective_lunch, n_trials=20)
lunch_param = study.best_trial.params

print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2021-07-23 12:23:52,033] A new study created in memory with name: cat_parameter_opt
[I 2021-07-23 12:23:57,683] Trial 0 finished with value: 66.1292241834929 and parameters: {'max_depth': 6, 'colsample_bylevel': 0.9901428612819833, 'subsample': 0.8123957592679836, 'min_child_samples': 62, 'max_bin': 246}. Best is trial 0 with value: 66.1292241834929.
[I 2021-07-23 12:24:01,945] Trial 1 finished with value: 66.72067151684455 and parameters: {'max_depth': 3, 'colsample_bylevel': 0.8116167224336399, 'subsample': 0.9063233020424546, 'min_child_samples': 62, 'max_bin': 413}. Best is trial 0 with value: 66.1292241834929.
[I 2021-07-23 12:24:06,964] Trial 2 finished with value: 70.96760989506923 and parameters: {'max_depth': 1, 'colsample_bylevel': 0.9939819704323989, 'subsample': 0.8827098485602951, 'min_child_samples': 25, 'max_bin': 254}. Best is trial 0 with value: 66.1292241834929.
[I 2021-07-23 12:24:10,903] Trial 3 finished with value: 65.7968117356165 and parameters: {'max_depth': 

Best Score: 65.7968117356165
Best trial: {'max_depth': 3, 'colsample_bylevel': 0.8608484485919076, 'subsample': 0.6673295021425665, 'min_child_samples': 46, 'max_bin': 287}


In [67]:
lunch_mae = 0
lunch_pred = np.array([0]*50).astype('float64')
n_fold = 12
kf = KFold(n_splits=n_fold, random_state=42, shuffle=True)

for train_index, test_index in kf.split(lunch_train):
    X_train, X_valid = lunch_train.iloc[train_index], lunch_train.iloc[test_index]
    y_train, y_valid = y_lunch[train_index], y_lunch[test_index]

    model = CatBoostRegressor(**lunch_param)

    evaluation = [(X_train, y_train), (X_valid, y_valid)]

    model.fit(X_train, y_train,
              eval_set=evaluation, 
              early_stopping_rounds=100,
              verbose=0)

    pred = model.predict(X_valid)
    lunch_mae += MAE(y_valid, pred)/n_fold
    
    lunch_pred += model.predict(lunch_test)/n_fold
    
print(lunch_mae)

64.20218143298891


In [51]:
def objective_dinner(trial: Trial) -> float: 
    params_cat = {
        "random_state": 42,
        "learning_rate": 0.05,
        "n_estimators": 10000,
        "verbose" : 1,
        "objective" : "MAE",
        "eval_metric" : "MAE",
        "max_depth": trial.suggest_int("max_depth", 1, 16),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.8, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }
    
    n_fold = 4
    kf = KFold(n_splits=n_fold, random_state=42, shuffle=True)
    mae_log = 0
    
    for train_index, test_index in kf.split(dinner_train):
        X_train, X_valid = dinner_train.iloc[train_index], dinner_train.iloc[test_index]
        y_train, y_valid = y_dinner[train_index], y_dinner[test_index]

        model = CatBoostRegressor(**params_cat)

        evaluation = [(X_train, y_train), (X_valid, y_valid)]

        model.fit(X_train, y_train,
                  eval_set=evaluation, 
                  early_stopping_rounds=100,
                  verbose=0)

        pred = model.predict(X_valid)
        mae= MAE(y_valid, pred)   
        
        mae_log += mae/n_fold
    
    return mae_log

In [52]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="cat_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective_dinner, n_trials=20)
dinner_param = study.best_trial.params

print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2021-07-23 11:40:32,678] A new study created in memory with name: cat_parameter_opt
Custom logger is already specified. Specify more than one logger at same time is not thread safe.[I 2021-07-23 11:40:36,180] Trial 0 finished with value: 56.092123296262514 and parameters: {'max_depth': 6, 'colsample_bylevel': 0.9901428612819833, 'subsample': 0.8123957592679836, 'min_child_samples': 62, 'max_bin': 246}. Best is trial 0 with value: 56.092123296262514.
[I 2021-07-23 11:40:38,264] Trial 1 finished with value: 58.36617224597725 and parameters: {'max_depth': 3, 'colsample_bylevel': 0.8116167224336399, 'subsample': 0.9063233020424546, 'min_child_samples': 62, 'max_bin': 413}. Best is trial 0 with value: 56.092123296262514.
[I 2021-07-23 11:40:41,589] Trial 2 finished with value: 60.3366394991943 and parameters: {'max_depth': 1, 'colsample_bylevel': 0.9939819704323989, 'subsample': 0.8827098485602951, 'min_child_samples': 25, 'max_bin': 254}. Best is trial 0 with value: 56.092123296262514.


Best Score: 55.54418143665878
Best trial: {'max_depth': 6, 'colsample_bylevel': 0.9956170942525168, 'subsample': 0.821262629615515, 'min_child_samples': 5, 'max_bin': 205}


In [53]:
dinner_mae = 0
dinner_pred = np.array([0]*50).astype('float64')

for train_index, test_index in kf.split(dinner_train):
    X_train, X_valid = dinner_train.iloc[train_index], dinner_train.iloc[test_index]
    y_train, y_valid = y_dinner[train_index], y_dinner[test_index]

    model = CatBoostRegressor(**dinner_param)

    evaluation = [(X_train, y_train), (X_valid, y_valid)]

    model.fit(X_train, y_train,
              eval_set=evaluation, 
              early_stopping_rounds=100,
              verbose=0)

    pred = model.predict(X_valid)
    dinner_mae += MAE(y_valid, pred)/n_fold
    
    dinner_pred += model.predict(dinner_test)/n_fold
    
print(dinner_mae)

53.30713963391145


In [68]:
submission = pd.read_csv('../data/sample_submission.csv')

submission.iloc[:,1] = lunch_pred
submission.iloc[:,2] = dinner_pred
submission.head()

,일자,중식계,석식계
0,2021-01-27,988.245069,338.878168
1,2021-01-28,921.191764,425.128727
2,2021-01-29,628.100691,225.429082
3,2021-02-01,1309.418597,539.132832
4,2021-02-02,1063.044738,488.659505


In [63]:
submission.iloc[:,1:].mean()

중식계    971.967640
석식계    475.127619
dtype: float64

In [59]:
def compare_ans(DIR):
    answer = pd.read_csv(DIR)

    lunch_answer = np.array(answer.iloc[:,1])
    dinner_answer = np.array(answer.iloc[:,2])
    
    lunch_MAE = abs(submission.iloc[:,1] - lunch_answer).mean()
    dinner_MAE = abs(submission.iloc[:,2] - dinner_answer).mean()
    
    print("lunch_MAE : ", lunch_MAE)
    print("dinner_MAE : ", dinner_MAE)
    print("total_MAE : ", (lunch_MAE+dinner_MAE)/2)

In [60]:
import glob

files = glob.glob('../submission/*')

for file in files[:-1]:
    print(file, compare_ans(file), end="\n\n" )

lunch_MAE :  14.502630285785663
dinner_MAE :  20.716353899615097
total_MAE :  17.60949209270038
../submission\20210624_pycaret_(4-2)_제출해봐야할거.csv None

lunch_MAE :  46.6653142924422
dinner_MAE :  29.062117930950215
total_MAE :  37.863716111696206
../submission\20210625_cat.csv None

lunch_MAE :  40.48944839591298
dinner_MAE :  23.722537308464027
total_MAE :  32.10599285218851
../submission\20210625_cat_10fold.csv None

lunch_MAE :  39.19760242344971
dinner_MAE :  20.72612936904833
total_MAE :  29.96186589624902
../submission\20210627_cat_15_KJH.csv None

lunch_MAE :  59.89625302543101
dinner_MAE :  45.495865556589436
total_MAE :  52.69605929101022
../submission\20210627_cat_after_2020.csv None

lunch_MAE :  39.19760242344971
dinner_MAE :  20.72612936904833
total_MAE :  29.96186589624902
../submission\20210627_cat_KJH.csv None

lunch_MAE :  39.19760242344971
dinner_MAE :  20.72612936904833
total_MAE :  29.96186589624902
../submission\20210627_pycaret_KJH.csv None

lunch_MAE :  28.9457297

# 저장

In [64]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission.to_csv(f'../submission/{today}_optuna_cat.csv', index =False)

오늘 날짜 : 20210723


In [146]:
hyperOpt = pd.read_csv('../submission/20210722_hyperOpt_lgbm.csv')

In [150]:
submission_mix = submission.copy()

In [155]:
submission_mix.iloc[:,1] = hyperOpt.iloc[:,1]

In [160]:
submission_mix.head(4)

,일자,중식계,석식계
0,2021-01-27,1006.208305,308.383648
1,2021-01-28,924.785410,416.512028
2,2021-01-29,634.318068,216.336773
3,2021-02-01,1293.689435,529.224504


In [161]:
submission_mix.to_csv(f'../submission/{today}_mix_lgbm.csv', index =False)